<a href="https://colab.research.google.com/github/indhu68/Intro_to_DL_Project/blob/main/RTML_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn
import matplotlib.pyplot as plt


In [2]:
# Load data

data = pd.read_csv('https://raw.githubusercontent.com/indhu68/Intro_to_DL_Project/main/Kasungu_Telemetry_Pts_Oct23.csv', parse_dates=['Time.Stamp'])
data = data[["Index","Tag", "Latitude", "Longitude", "Time.Stamp"]]
data['Time.Stamp'] = pd.to_datetime(data['Time.Stamp'])
data.sort_values(by=['Tag', 'Time.Stamp'], inplace=True)

In [3]:
# Calculate speed and distances
epsilon = 1e-5  # Small constant to avoid division by zero
data['Time_diff'] = data.groupby('Tag')['Time.Stamp'].diff().dt.total_seconds().fillna(0)
data['Lat_diff'] = data.groupby('Tag')['Latitude'].diff().fillna(0)
data['Lon_diff'] = data.groupby('Tag')['Longitude'].diff().fillna(0)
data['Speed'] = np.sqrt(data['Lat_diff']**2 + data['Lon_diff']**2) / (data['Time_diff'] + epsilon)

In [4]:
# Handle potential infinite or NaN values
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(data.mean(), inplace=True)  # Fill NaNs with the mean of the column


In [5]:
# Normalize features
scaler = MinMaxScaler()
data[['Latitude', 'Longitude', 'Lat_diff', 'Lon_diff', 'Speed']] = scaler.fit_transform(
    data[['Latitude', 'Longitude', 'Lat_diff', 'Lon_diff', 'Speed']]
)

In [6]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps + 1
        if end_ix > len(data):
            break
        seq_x = data.iloc[i:end_ix-1].to_numpy()
        seq_y = data.iloc[end_ix-1][['Latitude', 'Longitude']].to_numpy()
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Prepare sequences
grouped = data.groupby('Tag')
X, y = [], []
for _, group in grouped:
    sequences = create_sequences(group[['Latitude', 'Longitude', 'Lat_diff', 'Lon_diff', 'Speed']], n_steps=5)
    X.append(sequences[0])
    y.append(sequences[1])

X = np.concatenate(X)
y = np.concatenate(y)


In [7]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [9]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(5,32)
        self.lstm = nn.LSTM(input_size=32, hidden_size=128, num_layers=3, batch_first=True)
        self.fc = nn.Linear(128, 2)

    def forward(self, x):
        x = self.fc1(x)
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

model = LSTMModel().to('cuda')
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Prepare data loaders
from torch.utils.data import DataLoader, TensorDataset
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=True)

In [11]:

def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

# Prepare data loaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=True)

# Train the model
train(model, train_loader, criterion, optimizer)


Epoch 1, Loss: 0.0007912141111794672
Epoch 2, Loss: 2.5395743895865778e-05
Epoch 3, Loss: 1.9803334269354418e-05
Epoch 4, Loss: 1.7360332705081658e-05
Epoch 5, Loss: 1.5375103593761198e-05
Epoch 6, Loss: 1.4579710808357847e-05
Epoch 7, Loss: 1.3541012193516084e-05
Epoch 8, Loss: 1.3177556323283208e-05
Epoch 9, Loss: 1.2885541764452591e-05
Epoch 10, Loss: 1.2439723759068484e-05


In [12]:
import torch

def evaluate(model, dataloader, criterion, scaler):
    model.eval()
    total_loss = 0
    sample_count = 0

    min_lat_lon = scaler.data_min_[:2]  # Assuming these are lat and lon
    range_lat_lon = scaler.data_range_[:2]  # Assuming these are lat and lon ranges

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            total_loss += loss.item()

            if sample_count < 10:
                actuals = y_batch.cpu().numpy()
                predicteds = predictions.cpu().numpy()

                # Apply inverse scaling to latitude and longitude
                actuals[:, 0:2] = actuals[:, 0:2] * range_lat_lon + min_lat_lon
                predicteds[:, 0:2] = predicteds[:, 0:2] * range_lat_lon + min_lat_lon

                for actual, predicted in zip(actuals, predicteds):
                    print(f'Actual: {actual[:2]}, Predicted: {predicted[:2]}')
                    sample_count += 1
                    if sample_count >= 10:
                        break

    print(f'\nAverage Loss: {total_loss / len(dataloader)}')


In [13]:
evaluate(model, test_loader, criterion,scaler)

Actual: [-12.991917  33.272797], Predicted: [-12.988714  33.27226 ]
Actual: [-12.911055  33.23379 ], Predicted: [-12.907994  33.229473]
Actual: [-12.900917  33.197685], Predicted: [-12.900828  33.198223]
Actual: [-13.126538  33.124836], Predicted: [-13.125088  33.12282 ]
Actual: [-13.078202  33.061714], Predicted: [-13.078074  33.060375]
Actual: [-12.857592  33.105824], Predicted: [-12.860061  33.10612 ]
Actual: [-12.912452  33.309772], Predicted: [-12.907939  33.31548 ]
Actual: [-12.849057  33.20437 ], Predicted: [-12.849726  33.204796]
Actual: [-12.687453  33.128994], Predicted: [-12.683083  33.130253]
Actual: [-13.26807  33.11822], Predicted: [-13.273646  33.12118 ]

Average Loss: 9.571963517916166e-06


In [14]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(5, 32)
        self.rnn = nn.RNN(input_size=32, hidden_size=128, num_layers=3, batch_first=True)
        self.fc = nn.Linear(128, 2)

    def forward(self, x):
        x = self.fc1(x)
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])

rnn_model = RNNModel().to('cuda')
criterion_rnn = nn.MSELoss()
optimizer_rnn = torch.optim.Adam(rnn_model.parameters(), lr=0.001)


In [16]:

def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

# Prepare data loaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=True)

# Train the model
train(model, train_loader, criterion, optimizer)




Epoch 1, Loss: 1.0397343869313592e-05
Epoch 2, Loss: 1.0326715467344159e-05
Epoch 3, Loss: 1.0313110918807723e-05
Epoch 4, Loss: 1.0217532660889936e-05
Epoch 5, Loss: 1.0113994587039063e-05
Epoch 6, Loss: 1.0055014537845111e-05
Epoch 7, Loss: 1.0025053697532125e-05
Epoch 8, Loss: 1.0003591006619068e-05
Epoch 9, Loss: 9.969816263541741e-06
Epoch 10, Loss: 9.936153184118159e-06


In [17]:
import torch

def evaluate(model, dataloader, criterion, scaler):
    model.eval()
    total_loss = 0
    sample_count = 0

    min_lat_lon = scaler.data_min_[:2]  # Assuming these are lat and lon
    range_lat_lon = scaler.data_range_[:2]  # Assuming these are lat and lon ranges

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            total_loss += loss.item()

            if sample_count < 10:
                actuals = y_batch.cpu().numpy()
                predicteds = predictions.cpu().numpy()

                # Apply inverse scaling to latitude and longitude
                actuals[:, 0:2] = actuals[:, 0:2] * range_lat_lon + min_lat_lon
                predicteds[:, 0:2] = predicteds[:, 0:2] * range_lat_lon + min_lat_lon

                for actual, predicted in zip(actuals, predicteds):
                    print(f'Actual: {actual[:2]}, Predicted: {predicted[:2]}')
                    sample_count += 1
                    if sample_count >= 10:
                        break

    print(f'\nAverage Loss: {total_loss / len(dataloader)}')

evaluate(model, test_loader, criterion,scaler)


Actual: [-12.850323  33.186596], Predicted: [-12.852572  33.185753]
Actual: [-13.053623  33.13765 ], Predicted: [-13.051231  33.140804]
Actual: [-12.695882  33.131416], Predicted: [-12.701886  33.13119 ]
Actual: [-12.87794   33.249634], Predicted: [-12.874328  33.24885 ]
Actual: [-13.178442  33.120758], Predicted: [-13.181641  33.12123 ]
Actual: [-13.298882  33.02378 ], Predicted: [-13.290944  33.020878]
Actual: [-12.251355  32.429443], Predicted: [-12.255322  32.428684]
Actual: [-13.268332  33.11803 ], Predicted: [-13.267344  33.1196  ]
Actual: [-13.082685  33.178646], Predicted: [-13.083469  33.177605]
Actual: [-12.940212  33.310917], Predicted: [-12.9409485  33.314392 ]

Average Loss: 8.898865550152265e-06


In [18]:
class GRUModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(5, 32)
        self.gru = nn.GRU(input_size=32, hidden_size=128, num_layers=3, batch_first=True)
        self.fc = nn.Linear(128, 2)

    def forward(self, x):
        x = self.fc1(x)
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

gru_model = GRUModel().to('cuda')
criterion_gru = nn.MSELoss()
optimizer_gru = torch.optim.Adam(gru_model.parameters(), lr=0.001)


In [19]:

def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

# Prepare data loaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=True)

# Train the model
train(model, train_loader, criterion, optimizer)



Epoch 1, Loss: 9.885533056138883e-06
Epoch 2, Loss: 9.987743456577838e-06
Epoch 3, Loss: 9.830288158335676e-06
Epoch 4, Loss: 9.915423344067564e-06
Epoch 5, Loss: 9.827278457642101e-06
Epoch 6, Loss: 9.7230846728799e-06
Epoch 7, Loss: 9.818640588804798e-06
Epoch 8, Loss: 9.723774378364372e-06
Epoch 9, Loss: 9.659902048588797e-06
Epoch 10, Loss: 9.678963574313023e-06


In [20]:
import torch

def evaluate(model, dataloader, criterion, scaler):
    model.eval()
    total_loss = 0
    sample_count = 0

    min_lat_lon = scaler.data_min_[:2]  # Assuming these are lat and lon
    range_lat_lon = scaler.data_range_[:2]  # Assuming these are lat and lon ranges

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            total_loss += loss.item()

            if sample_count < 10:
                actuals = y_batch.cpu().numpy()
                predicteds = predictions.cpu().numpy()

                # Apply inverse scaling to latitude and longitude
                actuals[:, 0:2] = actuals[:, 0:2] * range_lat_lon + min_lat_lon
                predicteds[:, 0:2] = predicteds[:, 0:2] * range_lat_lon + min_lat_lon

                for actual, predicted in zip(actuals, predicteds):
                    print(f'Actual: {actual[:2]}, Predicted: {predicted[:2]}')
                    sample_count += 1
                    if sample_count >= 10:
                        break

    print(f'\nAverage Loss: {total_loss / len(dataloader)}')

evaluate(model, test_loader, criterion,scaler)


Actual: [-13.088043  33.21084 ], Predicted: [-13.087469  33.21344 ]
Actual: [-12.840453  33.193604], Predicted: [-12.840797  33.195   ]
Actual: [-13.06406   33.181767], Predicted: [-13.066008  33.182106]
Actual: [-12.774588  33.067142], Predicted: [-12.774734  33.068142]
Actual: [-13.114002  33.024143], Predicted: [-13.113149  33.026085]
Actual: [-12.902275  33.176537], Predicted: [-12.902032  33.17683 ]
Actual: [-13.296782  33.033176], Predicted: [-13.29606  33.03174]
Actual: [-13.118703  33.13329 ], Predicted: [-13.116373  33.134953]
Actual: [-12.67906   33.207382], Predicted: [-12.682636  33.20732 ]
Actual: [-12.969095  33.13352 ], Predicted: [-12.968284  33.135742]

Average Loss: 8.751852609616865e-06


In [21]:
import torch
import torch.nn as nn

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim * 4, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        return self.fc_out(x[:, -1, :])

# Example initialization and setting up optimizer and loss
transformer_attention_model = TransformerModel(input_dim=5, num_heads=4, num_layers=3, hidden_dim=128, output_dim=2).to('cuda')
criterion_transformer = nn.MSELoss()
optimizer_transformer = torch.optim.Adam(transformer_attention_model.parameters(), lr=0.001)


In [22]:

def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

# Prepare data loaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=True)

# Train the model
train(model, train_loader, criterion, optimizer)



Epoch 1, Loss: 9.620678285499857e-06
Epoch 2, Loss: 9.559379725176703e-06
Epoch 3, Loss: 9.544881166290032e-06
Epoch 4, Loss: 9.59409381858708e-06
Epoch 5, Loss: 9.55689977527057e-06
Epoch 6, Loss: 9.540485501401853e-06
Epoch 7, Loss: 9.584064781229655e-06
Epoch 8, Loss: 9.584830845273088e-06
Epoch 9, Loss: 9.456047791330367e-06
Epoch 10, Loss: 9.44257934510419e-06


In [23]:
import torch

def evaluate(model, dataloader, criterion, scaler):
    model.eval()
    total_loss = 0
    sample_count = 0

    min_lat_lon = scaler.data_min_[:2]  # Assuming these are lat and lon
    range_lat_lon = scaler.data_range_[:2]  # Assuming these are lat and lon ranges

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            total_loss += loss.item()

            if sample_count < 10:
                actuals = y_batch.cpu().numpy()
                predicteds = predictions.cpu().numpy()

                # Apply inverse scaling to latitude and longitude
                actuals[:, 0:2] = actuals[:, 0:2] * range_lat_lon + min_lat_lon
                predicteds[:, 0:2] = predicteds[:, 0:2] * range_lat_lon + min_lat_lon

                for actual, predicted in zip(actuals, predicteds):
                    print(f'Actual: {actual[:2]}, Predicted: {predicted[:2]}')
                    sample_count += 1
                    if sample_count >= 10:
                        break

    print(f'\nAverage Loss: {total_loss / len(dataloader)}')

evaluate(model, test_loader, criterion,scaler)


Actual: [-12.845487  33.308167], Predicted: [-12.841338  33.308952]
Actual: [-12.946897  33.343624], Predicted: [-12.946567  33.34222 ]
Actual: [-12.874117  33.247982], Predicted: [-12.872089  33.248318]
Actual: [-12.805455  33.20787 ], Predicted: [-12.803793  33.21308 ]
Actual: [-12.835657  33.13309 ], Predicted: [-12.836319  33.132877]
Actual: [-12.932943  33.25104 ], Predicted: [-12.933314  33.25215 ]
Actual: [-12.958168  33.223373], Predicted: [-12.969971  33.219498]
Actual: [-12.828597  33.105366], Predicted: [-12.82993   33.106277]
Actual: [-13.114282  33.12327 ], Predicted: [-13.119808  33.134727]
Actual: [-13.241723  33.1287  ], Predicted: [-13.241946  33.130436]

Average Loss: 8.57462619848259e-06
